In [1]:
import argparse
import json
import sys
from pathlib import Path

import optuna
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer
from sklearn.metrics import (
    classification_report,
    roc_auc_score,
    accuracy_score,
    roc_curve,
    precision_recall_curve,
    auc,
)

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler


from sklearn2pmml import sklearn2pmml
from sklearn2pmml.pipeline import PMMLPipeline
from skl2onnx import convert_sklearn, update_registered_converter
from skl2onnx.common._registration import get_shape_calculator
from skl2onnx.common.data_types import FloatTensorType, Int64TensorType, StringTensorType
from skl2onnx.common.shape_calculator import (
    calculate_linear_classifier_output_shapes,
    calculate_linear_regressor_output_shapes,
)

import matplotlib
# 画像をファイルに保存するだけなので、X サーバの無い環境でも動くようにバックエンドを固定
matplotlib.use("Agg")
import matplotlib.pyplot as plt
%matplotlib inline


from pathlib import Path
# from model_export import export_pipeline_to_onnx, export_pipeline_to_pmml

/Users/masatosasaki/Desktop/real-time-inference/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
DEFAULT_DATA =  "../data/Titanic-Dataset.csv"
DEFAULT_TEST_DATA =  "../data/Titanic-Dataset.csv"
DEFAULT_REPORT_DIR =  "../reports/titanic/random_forest"
DEFAULT_PMML_PATH =  "../model/titanic_random_forest.pmml"
DEFAULT_ONNX_PATH =  "../model/titanic_random_forest.onnx"


DEFAULT_DATA = Path(DEFAULT_DATA)
DEFAULT_TEST_DATA = Path(DEFAULT_TEST_DATA)
DEFAULT_REPORT_DIR = Path(DEFAULT_REPORT_DIR)
DEFAULT_PMML_PATH = Path(DEFAULT_PMML_PATH)
DEFAULT_ONNX_PATH = Path(DEFAULT_ONNX_PATH)


FEATURES = ["Pclass", "Sex", "Age", "SibSp", "Parch", "Fare", "Embarked"]
TARGET = "Survived"
NUMERIC_FEATURES = ["Age", "SibSp", "Parch", "Fare"]
CATEGORICAL_FEATURES = ["Pclass", "Sex", "Embarked"]
STRING_FEATURES = list(CATEGORICAL_FEATURES)

In [3]:
df = pd.read_csv(DEFAULT_DATA)
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
missing_cols = set(FEATURES + [TARGET]) - set(df.columns)

df = df.dropna(subset=[TARGET])
X_train_val = df[FEATURES].copy()
y_train_val = df[TARGET].astype(int)
for col in CATEGORICAL_FEATURES:
    X_train_val[col] = X_train_val[col].astype("string").fillna("missing").astype(str)

In [5]:
missing_cols = set(FEATURES + [TARGET]) - set(df.columns)

df = df.dropna(subset=[TARGET])
X_test = df[FEATURES].copy()
y_test = df[TARGET].astype(int)
for col in CATEGORICAL_FEATURES:
    X_test[col] = X_test[col].astype("string").fillna("missing").astype(str)

In [6]:
# 数値列の欠損には中央値で埋め、スケールを合わせる。
# StandardScaler を使うことで学習が安定しやすくなる。
numeric_transformer = Pipeline(
    steps=[
        ("imputer", SimpleImputer(strategy="median")),
        ("scaler", StandardScaler()),
    ]
)
categorical_transformer = Pipeline(
    steps=[
        ("encoder", OneHotEncoder(handle_unknown="ignore")),
    ]
)

# ColumnTransformer で列ごとの処理を分岐させる。
# こうしておけば PMML / ONNX へエクスポートするときも前処理を丸ごと含められる。
preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, NUMERIC_FEATURES),
        ("cat", categorical_transformer, CATEGORICAL_FEATURES),
    ]
)

In [8]:

test_size = 0.2
random_state = 0

def objective(trial: optuna.trial.Trial) -> float:
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 200, 800),
        "max_depth": trial.suggest_int("max_depth", 4, 20),
        "min_samples_split": trial.suggest_int("min_samples_split", 2, 8),
        "min_samples_leaf": trial.suggest_int("min_samples_leaf", 1, 6),
        "max_features": trial.suggest_categorical(
            "max_features", ["sqrt", "log2", None]
        ),
        "bootstrap": trial.suggest_categorical("bootstrap", [True, False]),
    }

    model = RandomForestClassifier(
        n_estimators=params["n_estimators"],
        max_depth=params["max_depth"],
        min_samples_split=params["min_samples_split"],
        min_samples_leaf=params["min_samples_leaf"],
        max_features=params["max_features"],
        bootstrap=params["bootstrap"],
        n_jobs=-1,
        random_state=random_state,
    )

    pipeline = Pipeline(steps=[("preprocess", preprocessor), ("model", model)])
    X_train, X_valid, y_train, y_valid = train_test_split(
        X_train_val,
        y_train_val,
        test_size=test_size,
        random_state=random_state,
        stratify=y_train_val,
    )
    pipeline.fit(X_train, y_train)
    proba = pipeline.predict_proba(X_valid)[:, 1]
    return roc_auc_score(y_valid, proba)

random_state = 42
n_trials = 1
study = optuna.create_study(
    direction="maximize",
    sampler=optuna.samplers.TPESampler(seed=random_state),
)
study.optimize(objective, n_trials=n_trials, show_progress_bar=False)
print(f"Best trial ROC AUC: {study.best_value:.4f}")
print(f"Best params: {study.best_params}")
best_params = study.best_params


[I 2025-11-09 23:55:12,179] A new study created in memory with name: no-name-9d12bbec-9591-4928-965d-916ce991f084
[I 2025-11-09 23:55:12,467] Trial 0 finished with value: 0.838866930171278 and parameters: {'n_estimators': 425, 'max_depth': 20, 'min_samples_split': 7, 'min_samples_leaf': 4, 'max_features': 'sqrt', 'bootstrap': True}. Best is trial 0 with value: 0.838866930171278.


Best trial ROC AUC: 0.8389
Best params: {'n_estimators': 425, 'max_depth': 20, 'min_samples_split': 7, 'min_samples_leaf': 4, 'max_features': 'sqrt', 'bootstrap': True}


In [9]:
best_params["n_estimators"]

425

In [10]:
model = RandomForestClassifier(
    n_estimators=best_params["n_estimators"],
    max_depth=best_params["max_depth"],
    min_samples_split=best_params["min_samples_split"],
    min_samples_leaf=best_params["min_samples_leaf"],
    max_features=best_params["max_features"],
    bootstrap=best_params["bootstrap"],
    n_jobs=-1,
    random_state=random_state,
)
pipeline = Pipeline(steps=[("preprocess", preprocessor), ("model", model)])
pipeline.fit(X_train_val, y_train_val)

,steps,"[('preprocess', ...), ('model', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('num', ...), ('cat', ...)]"
,remainder,'drop'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True


In [14]:

label = "external test"
preds = pipeline.predict(X_test)
proba = pipeline.predict_proba(X_test)[:, 1]

# Standard metrics
report = classification_report(y_test, preds)
fpr, tpr, _ = roc_curve(y_test, proba)
roc_auc = roc_auc_score(y_test, proba)
acc = accuracy_score(y_test, preds)

# ➕ PR-AUC computation
precision, recall, _ = precision_recall_curve(y_test, proba)
pr_auc = auc(recall, precision)

# Print results
print(f"\n=== Evaluation on {label} set ===")
print(report)
print(f"Accuracy: {acc:.4f}")
print(f"ROC AUC: {roc_auc:.4f}")
print(f"PR AUC:  {pr_auc:.4f}")


=== Evaluation on external test set ===
              precision    recall  f1-score   support

           0       0.86      0.95      0.90       549
           1       0.90      0.76      0.82       342

    accuracy                           0.88       891
   macro avg       0.88      0.85      0.86       891
weighted avg       0.88      0.88      0.87       891

Accuracy: 0.8754
ROC AUC: 0.9450
PR AUC:  0.9244


In [15]:
label_key = "external_test"

# === Save metrics (JSON) ===
metrics = {
    "roc_auc": roc_auc,
    "pr_auc": pr_auc,  # ➕ include PR-AUC
    "accuracy": acc,
    "support": len(y_test),
}
metrics_path = DEFAULT_REPORT_DIR / f"{label_key}_metrics.json"
with metrics_path.open("w") as f:
    json.dump(metrics, f, indent=2)

# === Plot ROC Curve ===
plt.figure(figsize=(6, 6))

plt.plot(fpr, tpr, label=f"ROC curve (AUC = {roc_auc:.4f})")

plt.plot([0, 1], [0, 1], "k--", label="Random")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title(f"ROC Curve - {label}")
plt.legend(loc="lower right")
plt.tight_layout()
roc_path = DEFAULT_REPORT_DIR / f"{label_key}_roc_curve.png"
plt.savefig(roc_path, dpi=120)
plt.close()  # Close after saving

# === Plot Precision–Recall Curve ===
plt.figure(figsize=(6, 6))
plt.plot(recall, precision, label=f"PR curve (AUC = {pr_auc:.4f})")
plt.xlabel("Recall")
plt.ylabel("Precision")
plt.title(f"Precision–Recall Curve - {label}")
plt.legend(loc="lower left")
plt.grid(True)
plt.tight_layout()
pr_path = DEFAULT_REPORT_DIR / f"{label_key}_pr_curve.png"
plt.savefig(pr_path, dpi=120)
plt.close()

# === Print results to console ===
print(f"\n=== Evaluation on {label} set ===")
print(report)
print(f"Accuracy: {acc:.4f}")
print(f"ROC AUC: {roc_auc:.4f}")
print(f"PR AUC:  {pr_auc:.4f}")
print(f"Saved reports to: {DEFAULT_REPORT_DIR.resolve()}")



=== Evaluation on external test set ===
              precision    recall  f1-score   support

           0       0.86      0.95      0.90       549
           1       0.90      0.76      0.82       342

    accuracy                           0.88       891
   macro avg       0.88      0.85      0.86       891
weighted avg       0.88      0.88      0.87       891

Accuracy: 0.8754
ROC AUC: 0.9450
PR AUC:  0.9244
Saved reports to: /Users/masatosasaki/Desktop/real-time-inference/reports/titanic/random_forest


In [16]:
sklearn2pmml(pipeline, DEFAULT_PMML_PATH, with_repr=True)

In [17]:
initial_type = []

# 数値列は float 型
for col in NUMERIC_FEATURES:
    initial_type.append((col, FloatTensorType([None, 1])))

# カテゴリ列は string 型
for col in CATEGORICAL_FEATURES:
    initial_type.append((col, StringTensorType([None, 1])))
onx = convert_sklearn(pipeline, initial_types=initial_type, target_opset=12)

with open(DEFAULT_ONNX_PATH, "wb") as f:
    f.write(onx.SerializeToString())